In [ ]:
# Import modules

import warnings

# import holoviews as hv
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.pyplot as plt
import matplotlib.ticker as tck
import numpy as np
import shapely
import pandas as pd
import pystac_client
import xarray as xr
import rioxarray as rio
import pathlib
from pathlib import Path
import pystac
import pystac_client
import geopandas as gpd

#import colormaps as cmaps
import pyam # https://pyam-iamc.readthedocs.io/en/latest/index.html
import matplotlib.colors as mcolors

from copy import deepcopy
from typing import List, Dict

# Import custom functionality
from coclicodata.drive_config import p_drive

# ignore warnings
warnings.filterwarnings("ignore")

In [ ]:
# Convert NUTS to parquet file

fp = Path(r"p:\11207608-coclico\FASTTRACK_DATA\XX_NUTS\NUTS_RG_01M_2021_3035.shp")

NUTS_data = gpd.read_file(fp)

NL_NUTS = NUTS_data.loc[NUTS_data.CNTR_CODE == 'NL']

NL_NUTS

In [ ]:
# Load LAU data

fp = Path(r"p:\11207608-coclico\FASTTRACK_DATA\XX_LAU\LAU_RG_01M_2020_3035.parquet")

LAU_data = gpd.read_parquet(fp)

NL_LAU = LAU_data.loc[LAU_data.CNTR_CODE == 'NL']
NL_LAU

In [ ]:
# Match LAU and NUTS using this work: https://edjnet.github.io/lau_centres/lau_nuts.html

LAU_NUTS_match = pd.read_csv(r'p:\11207608-coclico\FASTTRACK_DATA\XX_NUTS\lau_2020_nuts_2021_concordance_by_geo.csv')
# Drop unneeded columns
LAU_NUTS_match = LAU_NUTS_match.drop(columns=['gisco_id','country','lau_id','lau_name','population','area_km2','year'])
LAU_NUTS_match = LAU_NUTS_match.rename(columns={'fid': 'FID'})

In [ ]:
LAU_data

In [ ]:
# Merge the two dataframes
LAU_NUTS_data = LAU_data.merge(LAU_NUTS_match,on='FID')

# Reorder columns
# Retrieve column names
cols = LAU_NUTS_data.columns

# Move columns
cols = cols.insert([2],cols[-2:])

# Drop the old ones
cols = cols[:-2]

# Store in final form
LAU_NUTS_data = LAU_NUTS_data[cols]

# Write geodataframe to parquet
coclico_data_dir = p_drive.joinpath("11207608-coclico", "FASTTRACK_DATA")
ds_dir = coclico_data_dir.joinpath('XX_LAU')
out_file= ds_dir.joinpath('LAU_2020_NUTS_2021_01M_3035.parquet')

# altered_lau_data.to_parquet(out_file_altered)
LAU_NUTS_data.to_parquet(out_file)


In [ ]:
LAU_NUTS_data